In [1]:
import serial
import serial.tools.list_ports as list_ports
import time

### 寻找可用串口

In [2]:
port_list = list(list_ports.comports())
print(port_list)
if len(port_list) == 0:
    print("无可用串口")
else:
    for i in range(len(port_list)):
        print(port_list[i])

[<serial.tools.list_ports_common.ListPortInfo object at 0x000002D0F257AB20>, <serial.tools.list_ports_common.ListPortInfo object at 0x000002D0F2613EB0>]
COM3 - USB-SERIAL CH340 (COM3)
COM4 - USB-SERIAL CH340 (COM4)


In [27]:
list(list_ports.comports())[0]

In [23]:
print(list(list_ports.comports())[0][0])

COM3


In [28]:
print(list(list_ports.comports())[0][1])

USB-SERIAL CH340 (COM3)


In [29]:
print(list(list_ports.comports())[0][2])

USB VID:PID=1A86:7523 SER= LOCATION=1-10


### 测试参数

In [3]:
#端口
portx = "COM3"
#波特率，标准值之一：50,75,110,134,150,200,300,600,1200,
#1800,2400,4800,9600,19200,38400,57600,115200
bps = 115200
#超时设置,None：永远等待操作，0为立即返回请求结果，其他值为等待超时时间(单位为秒）
timex = 5
#bytesize,一字节长度
bs = serial.EIGHTBITS
#stopbits
sb = serial.STOPBITS_ONE
#parity
pty = serial.PARITY_NONE

#RTS/CTS
rtscts = False

### 更多的参数定义

In [4]:
BLE_NAME = "BLE_02"
BLE_HELLO = "HELLO WORLD!"

#工作模式
BLE_MODE_M = "M" #主模式
BLE_MODE_S = "S" #从模式
BLE_MODE_O = "O" #观察模式(不可连接,扫描广播数据)
BLE_MODE_B = "B" #广播模式(不可连接，广播用户自定义数据)
BLE_MODE_I = "I" #iBeacon(不可连接,广播特定数据,室内定位)
#发射功率
BLE_TP_N14dBm = 1 #-14dBm
BLE_TP_N11dBm = 2  #-11dBm
BLE_TP_N8dBm = 3  #-8dBm
BLE_TP_N5dBm = 4  #-5dBm
BLE_TP_N2dBm = 5  #-2dBm
BLE_TP_P2dBm = 6  #2dBm
BLE_TP_P4dBm = 7  #4dBm
BLE_TP_P8dBm = 8  #8dBm
#串口设置
BLE_UARTBPS = 115200 #50,75,110,134,150,200,300,600,1200,1800,2400,4800,9600,19200,38400,57600,115200
BLE_UARTDATE = 8     #数据位，bytesize
BLE_UARTSTOP = 1     #停止位
BLE_UARTPR = 0       #PARITY

#超时设置
TIMEX = 5

#广播速度
BLE_ADPTIM = 5  #单位：10ms

#链路匹配连接使能
BLE_LINKPASSEN = 0

#板载LED使能
BLE_LEDEN = 1

#从设备断连睡眠使能
BLE_SLAVESLPPEN = 0

#蓝牙最大输出使能
BLE_MAXPUT = 0

### 打印串口信息

In [5]:
try:
    ser = serial.Serial(portx, bps, timeout=5, parity=pty, bytesize=bs, stopbits=sb, rtscts=rtscts)
    print("串口详情参数: ",ser)
    print(ser.port) #port name
    print(ser.baudrate)
    print(ser.bytesize)
except Exception as e:
    print("-----Error-----", e)

串口详情参数:  Serial<id=0x2d0f2685dc0, open=True>(port='COM3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=5, xonxoff=False, rtscts=False, dsrdtr=False)
COM3
115200
8


In [8]:
ser.rts

True

In [9]:
ser.rts = False

In [10]:
ser.rts

False

In [11]:
ser.cts

False

### 读写测试

In [6]:
#命令检查函数
def ble_cmd_check(ser, ack):
    """ser: Serial object"""
    """ack: acknowledgement"""
    reply = ser.read_all()
    if ack in reply.decode("utf-8"):
        ##print("成功")
        return True
    else:
        ##print("失败，回信为：", reply.decode("utf-8"))
        return False

In [7]:
#写命令函数
import time

def ble_send_cmd(ser, cmd, ack):
    """ser: Serial object"""
    """cmd: cmd str with 'AT+'"""
    """ack: acknoledgement"""
    ser.reset_output_buffer()
    ser.reset_input_buffer()
    if ser.writable():
        ser.write((cmd+"\r\n").encode("utf-8"))
        time.sleep(0.1) #wait for 10 ms
        if ble_cmd_check(ser, ack):
            ##print("成功发送命令：", cmd)
            return True;
        else:
            print("失败发送命令：", cmd)
            return False
    else:
        print(ser.port, " is not writable")
        return False

In [8]:
if ser.writable():
    ser.reset_input_buffer()
    ser.reset_output_buffer()
    print("接收缓存字节数：", ser.inWaiting())
    print("发送缓存字节数：", ser.out_waiting)
    try:
        ser.write("+++a\r\n".encode("utf-8"))
    except Exception as e:
        print("--Writing Error--", e)
    #不能使用read_line,因为成功收到的数据为“b'\r\nOK\r\n'”
    time.sleep(0.01)
    reply = ser.read_all()
    print("接收缓存字节数：", ser.inWaiting())
    print("回复为：", reply)
    
else:
    print(ser.port, " is not writable")

接收缓存字节数： 0
发送缓存字节数： 0
接收缓存字节数： 0
回复为： b'a+ok\r\n'


In [9]:
if ser.writable():
    cmd = "+++a"
    #进入配置模式
    if ble_send_cmd(ser, cmd, "a+ok"):
        print("进入配置模式")
    else:
        print("失败进入配置模式")
else:
    print(ser.port, " is not writable")

失败发送命令： +++a
失败进入配置模式


In [16]:
ble_send_cmd(ser, "AT+ENTM", "OK")

True

In [17]:
ble_send_cmd(ser, "AT+HELLO?", "OK")

失败发送命令： AT+HELLO?


False

### BLE模块参数配置

In [18]:
temp = "%d" % (BLE_TP_P2dBm)
print(temp)

6


In [10]:
def ble_set_init(ser):
    """SER: Serial object"""
    #进入配置模式
    while ble_send_cmd(ser, "+++a", "a+ok") == False:
        if ble_send_cmd(ser, "+++a", "ERROR") == True: #已经在配置模式
            break
        time.sleep(0.1)
    
    #AT test
    if ble_send_cmd(ser, "AT", "OK") == True:
        print("AT OK")
    
    #蓝牙名称设置
    name_cmd = "AT+NAME="+BLE_NAME
    if ble_send_cmd(ser, name_cmd, "OK") == True:
        print(name_cmd, " OK")
    else:
        print(name_cmd, " ERROR")
    
    #开机欢迎语设定
    hello_cmd = "AT+HELLO=" + BLE_HELLO
    if ble_send_cmd(ser, hello_cmd, "OK") == True:
        print(hello_cmd, " OK")
    else:
        print(hello_cmd, " ERROR") 
    
    #发射功率设置
    ble_tp = "AT+TPL=%d" % BLE_TP_P2dBm
    if ble_send_cmd(ser, ble_tp, "OK") == True:
        print(ble_tp, " OK")
    else:
        print(ble_tp, " ERROR")
        
    #串口设置
    ble_uart = "AT+UART=%d,%d,%d,%d" % (BLE_UARTBPS, BLE_UARTDATE, BLE_UARTPR, BLE_UARTSTOP)
    if ble_send_cmd(ser, ble_uart, "OK") == True:
        print(ble_uart, " OK")
    else:
        print(ble_uart, " ERROR")
    
    #广播速度设置
    ble_adptim = "AT+ADPTIM=%d" % (BLE_ADPTIM)
    if ble_send_cmd(ser, ble_adptim, "OK"):
        print(ble_adptim, " OK")
    else:
        print(ble_adptim, " ERROR")
    
    #链路匹配连接设置
    if BLE_LINKPASSEN == 1:
        if ble_send_cmd(ser, "AT+LINKPASSEN=ON", "OK"):
            print("AT+LINKPASSEN=ON", " OK")
        else:
            print("AT+LINKPASSEN=ON", " ERROR")
    else:
        if ble_send_cmd(ser, "AT+LINKPASSEN=OFF", "OK"):
            print("AT+LINKPASSEN=OFF", " OK")
        else:
            print("AT+LINKPASSEN=OFF", " ERROR")
    
    #板载LED设置
    if BLE_LEDEN == 1:
        if ble_send_cmd(ser, "AT+LEDEN=ON", "OK"):
            print("AT+LEDEN=ON", " OK")
        else:
            print("AT+LEDEN=ON", " ERROR")
    else:
        if ble_send_cmd(ser, "AT+LEDEN=OFF", "OK"):
            print("AT+LEDEN=OFF", " OK")
        else:
            print("AT+LEDEN=OFF", " ERROR")
    #从设备断连睡眠设置
    if BLE_SLAVESLPPEN == 1:
        if ble_send_cmd(ser, "AT+SLAVESLEEPEN=ON", "OK"):
            print("AT+SLAVESLEEPEN=ON", " OK")
        else:
            print("AT+SLAVESLEEPEN=ON", " ERROR")
    else:
        if ble_send_cmd(ser, "AT+SLAVESLEEPEN=OFF", "OK"):
            print("AT+SLAVESLEEPEN=OFF", " OK")
        else:
            print("AT+SLAVESLEEPEN=OFF", " ERROR")
    
    #蓝牙最大输出设置
    if BLE_MAXPUT == 1:
        if ble_send_cmd(ser, "AT+SLAVESLEEPEN=ON", "OK"):
            print("AT+SLAVESLEEPEN=ON", " OK")
        else:
            print("AT+SLAVESLEEPEN=ON", " ERROR")
    else:
        if ble_send_cmd(ser, "AT+SLAVESLEEPEN=OFF", "OK"):
            print("AT+SLAVESLEEPEN=OFF", " OK")
        else:
            print("AT+SLAVESLEEPEN=OFF", " ERROR")
    
    ble_mode = "AT+MODE=%c" % BLE_MODE_S
    if ble_send_cmd(ser, ble_mode, "OK"):
        print(ble_mode, " OK")
    else:
        print(ble_mode, " ERROR")
    

In [11]:
ble_set_init(ser)

失败发送命令： +++a
AT OK
AT+NAME=BLE_02  OK
AT+HELLO=HELLO WORLD!  OK
AT+TPL=6  OK
AT+UART=115200,8,0,1  OK
AT+ADPTIM=5  OK
AT+LINKPASSEN=OFF  OK
AT+LEDEN=ON  OK
AT+SLAVESLEEPEN=OFF  OK
AT+SLAVESLEEPEN=OFF  OK
AT+MODE=S  OK


### BLE模块发送接收处理

In [12]:
def ble_inquiry_cmd(ser, cmd):
    """
    ser: Serial object
    cmd: cmd str with 'AT+'
    """
    while ble_send_cmd(ser, "+++a", "a+ok") == False:
        if ble_send_cmd(ser, "+++a", "ERROR") == True: #已经在配置模式
            break
        time.sleep(0.1)
    ser.reset_output_buffer()
    ser.reset_input_buffer()
    if ser.writable():
        ser.write((cmd+"\r\n").encode("utf-8"))
        time.sleep(0.1) #wait for 10 ms
        reply = ser.read_all()
        print("Reply is: ", reply.decode("utf-8"))
    else:
        print(ser.port, " is not writable")
        return False

In [13]:
def ble_set_cmd(ser, cmd, ack):
    """
    ser: Serial object
    cmd: cmd str with 'AT+'
    ack: acknowledgement str
    """
    while ble_send_cmd(ser, "+++a", "a+ok") == False:
        if ble_send_cmd(ser, "+++a", "ERROR") == True: #已经在配置模式
            break
        time.sleep(0.1)
    if ble_send_cmd(ser, cmd, ack):
        return True
    else:
        return False

In [14]:
#未考虑buffer溢出的问题

def ble_send_data(ser, data, encoding):
    """
    ser: Serial object
    data: str
    encoding: encoding format
    """
    if ser.writable():
        ser.reset_output_buffer()
        if encoding == "":
            ser.write(data)
        else:
            ser.write(data.encode(encoding))
    else:
        print("Fail to write: ", data)
        
def ble_rece_data(ser, decoding):
    """
    ser: Serial object
    decoding: decoding format
    """
    if decoding == "":
        res = ser.read_all()
    else:
        res = ser.read_all().decode(decoding)
    ser.reset_input_buffer()
    return res

In [ ]:
ser.read

In [ ]:
ble_send_data(ser, "LED0:OFF", "utf-8")

In [ ]:
ble_inquiry_cmd(ser, "AT+LINK?")

In [ ]:
ble_inquiry_cmd(ser, "AT+NAME?")

In [15]:
ble_set_cmd(ser, "AT+DISCONN", "OK")

True

In [18]:
ble_inquiry_cmd(ser, "AT+LINK?")

失败发送命令： +++a
Reply is:  
PeerAddr:84C2E4408C1A                     
Rssi:-55 dBm                     
+LINK:OnLine                     

OK



In [17]:
ble_inquiry_cmd(ser, "AT+MAC?")

Reply is:  
+MAC:84C2E4408770

OK



In [16]:
ble_set_cmd(ser, "AT+ENTM", "OK")

失败发送命令： +++a


True

### 测试手机蓝牙的结果

In [27]:
print(ble_rece_data(ser, "utf-8"))

hello world!


### 测试主从模透的结果

In [22]:
print(ble_rece_data(ser, "utf-8"))

ATK-BLE01 TEST 0



In [17]:
ble_send_data(ser, "test2", "utf-8")

### 正则表达式测试

In [31]:
import re

In [36]:
pattern = r'AT\+[A-Z]*'
string = "AT+name+"
print(re.fullmatch(pattern, string))

None


In [37]:
pattern = r'AT\+[A-Z]*'
string = "AT+NAME"
print(re.fullmatch(pattern, string))

<re.Match object; span=(0, 7), match='AT+NAME'>


In [39]:
pattern = r'AT\+[A-Z]*\?'
string = "AT+NAME?"
print(re.fullmatch(pattern, string))

<re.Match object; span=(0, 8), match='AT+NAME?'>


### 通信语句解析

In [6]:
print("sdihisd\rsdasds")

sdasds


In [1]:
import re

def dataParser(data):
    """
    data: str, should match 'row+col+value'
    return: 
    return None if the format of data is wrong
    return (row, col, val)
    """
    data = data.replace("\r", "")
    if re.fullmatch(r'[0-9]+\+[0-9]+\+[0-9]+', data) == None:
        return None
    data_list = data.split('+')
    return (int(data_list[0]), int(data_list[1]), int(data_list[2]))

In [19]:
print(dataParser("+232+2323\r"))

None


In [20]:
row, col, val = dataParser("232+2323+2323")

In [21]:
print(row, col, val)

232 2323 2323


In [2]:
print(dataParser("0+1+2706"))

(0, 1, 2706)
